In [41]:
import wandb

wandb.login()

True

In [42]:
config = dict(
    epochs = 5,
    classes = 10,
    kernels = [16,32],
    batch_size = 128,
    learning_rate = 0.005,
    dataset = "MNIST",
    architecture = "CNN"
)

In [43]:
def model_pipleline(hyperparameters):
    
    # tell wandb to get started
    with wandb.init(project = 'pytorch-demo', config = hyperparameters):
        config = wandb.config
    # make the model, data, and optimization problem
        model, train_loader, test_loader, criterion, optimizer = make(config)
        print(model)
    # and use them to train the model
        train(model, train_loader, criterion, optimizer, config)
    # and test its final performance
        test(model, test_loader)
    
    return model 
    

In [44]:
def make(config):
    # Make the data
    train, test = get_data(train=True), get_data(train = False)
    train_loader = make_loader(train, batch_size = config.batch_size)
    test_loader = make_loader(test, batch_size = config.batch_size)
    
    #Make the model
    model = ConvNet(config.kernels, config.classes).to(device)
    
    #Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = config.learning_rate)
    
    return model, train_loader, test_loader, criterion, optimizer
    

In [45]:
import torch
import torchvision
from torchvision import transforms

def get_data(slice=5, train=True):
    # Load MNIST dataset với transform là ToTensor
    full_dataset = torchvision.datasets.MNIST(
        root=".", 
        train=train, 
        transform=transforms.ToTensor(), 
        download=True
    )
    
    # Slicing dataset theo chỉ số step = slice
    sub_dataset = torch.utils.data.Subset(
        full_dataset, indices=range(0, len(full_dataset), slice)
    )
    
    return sub_dataset

def make_loader(dataset, batch_size, num_workers=4):
    # Tạo DataLoader từ dataset
    loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=True,
        pin_memory=True,
        num_workers=num_workers
    )
    
    return loader


In [46]:
import torch
import torch.nn as nn

class ConvNet(nn.Module):
    def __init__(self, kernels, classes=10):
        super(ConvNet, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, kernels[0], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(kernels[0], kernels[1], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Fully connected layer
        self.fc = nn.Linear(7 * 7 * kernels[-1], classes)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)  # Flatten
        out = self.fc(out)
        return out


SyntaxError: invalid syntax (4072768612.py, line 7)

In [34]:
def train(model, loader, criterion, optimizer, config):
    wandb.watch(model, criterion, log= "all", log_freq=10)
    
    total_batches = len(loader) * config.epochs
    example_ct = 0
    batch_ct =0
    
    for epoch in tqdp(range(config.epochs)):
        for _, (images, labesl) in enumerate(loader):
            
            loss = train_batch(images, labels, model,optimizer,criterion)
            example_ct += len(images)
            batch_ct +=1
            if ((batch_ct+1)%25) == 0:
                train_log(loss,example_ct,epoch)\
                    
def train_batch(images, labels, model, optimizer,criterion):
    images, labels = images.to(device), labels.to(device)
    
    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    #Backward pass
    optimizer.zero_grad()
    loss.backward()
    #Step with optimizer
    optimizer.step()
    
    return loss

IndentationError: expected an indented block (1226189411.py, line 17)